# Frequency and time-frequency sensors analysis


The objective is to show you how to explore the spectral content
of your data (frequency and time-frequency). Here we'll work on Epochs.

    Authors: Britta Westner, Alexandre Gramfort, Stefan Appelhoff, Richard Höchenberger, Denis A. Engemann
    License: BSD (3-clause)

In [ ]:
%matplotlib inline
import os

import numpy as np
import matplotlib.pyplot as plt

import mne
from mne.time_frequency import tfr_morlet

As before, we again set our paths. Then we load the epochs data we have saved.


In [ ]:
mne.set_log_level('error')

# Change the following path to where the folder ds000117 is on your disk
data_path = os.path.expanduser("~/Documents/teaching/practical_meeg_2022_data/ds000117")

epochs_fname = os.path.join(data_path,
    'derivatives/meg_derivatives/sub-01/ses-meg/meg/sub-01_ses-meg_task-facerecognition_run-01_proc-sss-epo.fif')

In [ ]:
epochs = mne.read_epochs(epochs_fname, proj=True)

In [ ]:
epochs.info

Frequency analysis
------------------

We start by exploring the frequency content of our epochs.



Let's first check out the power spectrum of the different channel types by averaging across epochs.

<li> The "bandwidth" parameter controls the spectral resolution of the multitaper. You can increase the resolution by chosing a narrower bandwidth at the cost of longer computation time.</li>
<li> You can also set method to `'welch'`, which will compute a power spectrum without using multi tapers.</li>


In [ ]:
# Let's first compute the power spectrum

epochs_psd = epochs.compute_psd(method='multitaper', fmin=2., fmax=40., bandwidth=2.)

In [ ]:
epochs_psd

In [ ]:
%matplotlib qt
epochs_psd.plot();

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
    <li> What is the name of the EEG channel with the highest power at high frequencies?</li>
    <li> What do the topographies at (roughly) 8-12 Hz look like?</li>
    </ul>
</div>

We can also look at the topographies of this power spectrum, e.g. for different frequency bands. Here, we have to specify a channel type!

In [ ]:
%matplotlib inline
bands = {'Theta (4-8 Hz)': (4, 8), 'Alpha (8-12 Hz)': (8, 12), 
         'Beta (12-30 Hz)': (12, 30), 'Gamma (30-05 Hz)': (30, 40)}
epochs_psd.plot_topomap(ch_type='mag', bands=bands, normalize=False, cmap='viridis');

The output of `compute_psd()` is a `Spectrum` object. We can index it similarly to `Epochs`.

Sometimes it can be interesting to consider the relative power, defined as the power in a given band divided by the total power. To explore this option, we can set the `normalize` parameter to `True`.

In [ ]:
epochs_psd['face'].plot_topomap(ch_type='mag', bands=bands, normalize=True, cmap='viridis');
epochs_psd['scrambled'].plot_topomap(ch_type='mag', bands=bands, normalize=True, cmap='viridis');

## Time-frequency analysis: power and inter-trial coherence

We now compute time-frequency representations (TFRs) from our Epochs.
We'll look at power and inter-trial coherence (ITC).

To this we'll use the function `mne.time_frequency.tfr_morlet`
but you can also use `mne.time_frequency.tfr_multitaper`
or `mne.time_frequency.tfr_stockwell`.

In [ ]:
# define frequencies of interest (log-spaced)
freqs = np.logspace(*np.log10([4, 30]), num=20)
n_cycles = 5.  
power, itc = tfr_morlet(epochs, freqs=freqs, n_cycles=n_cycles, use_fft=True,
                        return_itc=True, decim=3, n_jobs=1)

In [ ]:
 # crop to remove edge artifacts
power.crop(-0.2, 1.6) 
itc.crop(-0.2, 1.6) 

Inspect power
-------------

Let's look at the time-frequency spectra of all channels.


<div class="alert alert-info"><h4>Note</h4><p>The generated figures are interactive. In the topo you can click
    on an image to visualize the data for one sensor.
    You can also select a portion in the time-frequency plane to
    obtain a topomap for a certain time-frequency region.</p></div>



In [ ]:
# Some setting for our baseline, which will be applied to the plots
baseline_mode = 'logratio' 
baseline = (-0.2, 0)

In [ ]:
%matplotlib qt
power.plot_topo(baseline=baseline, mode=baseline_mode, title='Average power');

In [ ]:
%matplotlib inline
power.plot([83], baseline=baseline, mode=baseline_mode, title=power.ch_names[83]);

We can look at topographies again - 

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(7, 4))
plot_freqs = [(4, 7), (8, 12), (15, 30)]
titles = ['Theta', 'Alpha', 'Beta']

for ax, freq, title in zip(axes, plot_freqs, titles):
    power.plot_topomap(ch_type='mag', tmin=0.5, tmax=1.0, 
                   fmin=freq[0], fmax=freq[1],
                   baseline=baseline, mode=baseline_mode, 
                   axes=ax, show=False, contours=1)
    ax.set_title(title)

mne.viz.tight_layout()
plt.show()

Joint Plot
----------
You can also create a joint plot showing both the aggregated TFR
across channels and topomaps at specific times and frequencies to obtain
a quick overview regarding oscillatory effects across time and space.



In [ ]:
power.plot_topo?

In [ ]:
power.plot_joint(baseline=baseline, mode='mean', tmin=None, tmax=None,
                 timefreqs=[(0.2, 4.), (1., 10.)]);

## Compute FWHM for our wavelets

We defined our wavelets based on _number of cycles_. Mike X Cohen (Cohen 2019, NeuroImage (199, p. 81-86)) argues that it is better to define the full-width-at-half-maximum (FWHM) of the Morlet wavelet.

You might remember that our wavelet of `n_cycles` gets multiplied with a Gaussian taper that dampens the wavelet towards the edges. Thus, the FWHM is a better estimate of the temporal smoothing than the full length of `n_cycles`.

Let's see how to do this:

In [ ]:
# First let's check our cycles per frequency - we had specified them above:
freqs, n_cycles  # this is the number of cycles we asked for per frequency

The formula to retrieve the FWHM is (Cohen 2019, eq. 4):

$ FWHM = \frac{n  \sqrt(2  \ln 2)}{\pi * f} \enspace,$

where $n$ is the number of cycles, $f$ is the frequency, and $\ln$ denotes the natural logarithm.

Let's convert this equation into a little function!

In [ ]:
def get_fwhm_morlet(n_cycles, freq):
    """Estimate the FWHM of a Morlet wavelet."""

    fwhm = (n_cycles * np.sqrt(2 * np.log(2))) / (np.pi * freq)
    return fwhm


Now let's estimate what length of the FWHM of our Morlet wavelets was in seconds:

In [ ]:
for freq in freqs:
    
    # estimate fwhm
    fwhm = get_fwhm_morlet(n_cycles, freq)

    # print it
    print('FWHM at %.3f Hz was %.3f s' % (freq, fwhm))

Inspect ITC
-----------



In [ ]:
%matplotlib qt
itc.plot_topo(title='Inter-Trial coherence', vmin=0., vmax=1., cmap='Reds');

In [ ]:
%matplotlib inline
itc.plot_joint(baseline=baseline, mode='mean', tmin=None, tmax=None,
                 timefreqs=[(0.2, 4.), (1., 10.)]);